
<br> Theme of the material:**Time series analysis for hurricane prediction using LSTMs** <br>
Prepared by: *Getachew Workineh Gella* <br>
Paris Lodron University of Salzburg (PLUS), Department of Applied Geoinformatics<br>
email: getachewworkineh.gella@plus.ac.at

**Note!:** *Some commented cells are optional based on availability of data and implemntation strategy*
#### Loading Necessary packages for data reading, exploration and sequence modeling

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import seaborn as sbn
import matplotlib.pyplot as plt
import datetime
import random
from sklearn import preprocessing
from sklearn.model_selection import train_test_split  # if sample split is not done by user defined code
from tensorflow.keras.layers import Input, LSTM, Dropout, Bidirectional, BatchNormalization, Dense
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

#### Read the data
It should be noted that while reading the data using pandas dataframe approach, it is possible to manage nodata values on the flight or remove it after reading the data
For this excercise I have opted to tell the datafram reader to consider -999 as No data value

In [3]:
path = '/content/drive/MyDrive/plus-deepdata-social-good-master/plus-deepdata-social-good-master/atlantic.csv'
PD = pd.read_csv(path, na_values=-999)   # -999

#### Inspect the attribute names(column names) and respective data types.
This will enable enable to select proper data conversion options in further processing steps.
It shoud be noted that in pandas dataframe string objects are considered as object data types

In [40]:
print('The number of attributes in the dataset is : {}'.format(len(list(PD.keys()))))
col_names = list(PD.keys())
print('Attribute -> data type')
for name in col_names:
    print('{} -> {}'.format(name, PD[name].dtype))

### Inspect the first five and the last five rows with full columns.
<div style="text-align: justify"> This gives overll picture of data. As can be seen from the table displayed below, the first few rows
are dominated by incomplete observations except for latitude, longitue of the herican cenetrs. As can be seen from Herican Unique ID and Unique name count, names are less than Unique IDS. This is mainly
because of repeatedly happening herricaens happening at some geographies were given the same names but different ids and some hericanes were given a name of UnKnown.Therefore, as every trajectory monitoring is based on tagged unique herican name, in the feature engineering/preparation section, unique IDS will use to organize the data.   </div>


In [5]:
PD.head()

In [4]:
# inspect the data structure of the last five rows with all columns
PD.tail()

In [41]:
# count names and IDS of the hericanes
ids = np.unique(PD['ID'])
print('Number of total hurican unique IDS: {}'.format(len(ids)))
names = np.unique(PD['Name'])
print('Number of total hurican unique names: {}'.format(len(names)))

# Data conversion and cleaning
<div style="text-align: justify"> As can be seen from the tables displayed the following observations were taken into consideration <br>
1. Observations with massive incomplete data(have to be excluded from analysis) <br>
    2. Some rows whith some missing values were imputed with mean values of observations from the same herican. It should ne noted that, this should betaken with great caution as it will introduce extra noise to the data <br>
    3. Latitude and longitude values which are the main interest of this analysis were given as textual information whith hemispheric information(North/south, west/east).Using this information, the coordinates were converted to float longitude and latutidude. X coordinate in west and Y coordinate in south hemispher were converted to negative values. this gives a universal/uniform representation of coordinates <br>
    4. Date value is provided as string literal (YYMMDD) and then converted to Day of the Year (DOY) format <br>
    5. The Time value is converted from thousand notation to normal 24 hours scale which is more meaningfull </div>



In [9]:
def manage_long(df):
    '''
    Converts text longitude into numeric longitude
    the west is converted into negative values while the east is converted to positive values
    df: a data frame containing longitude value
    returns Longitude value with numeric representation without hemispheric information
    '''
    dff = df.copy()
    long = np.array(dff['Longitude'])

    for i in range(long.shape[0]):
        if "W" in long[i]:
            long[i] = float(long[i].replace('W',''))*-1
        elif 'E' in long[i]:
            long[i] = float(long[i].replace('E',''))*1
        else:
            raise ValueError('The value has no direction indicator')
    return long

In [10]:
def manage_lat(df):
    '''
    Converts text latitude into numeric latitude
    the South is converted into negative values while the North is converted to positive values
    df: a data frame containing Latitude value
    returns Latitude value with numeric representation without hemispheric information
    '''
    dff = df.copy()
    lat = np.array(dff['Latitude'])

    for i in range(lat.shape[0]):
        if "S" in lat[i]:
            lat[i] = float(lat[i].replace('S',''))*-1
        elif "N" in lat[i]:
            lat[i] = float(lat[i].replace('N',''))*1
        else:
            raise ValueError('The value has no direction indicator')
    return lat

##  Remove a subset of the dataset which have not sufficient observations and check presence of nodata values
As can be seen from the cell output below, even after droping a subset of the dataset which have nodata value there are still 80 records with nodata value for some columns/attributes.

In [49]:
PD_n = PD.loc[43106:]
PD_n.isna().sum()

## Convert latitude and longitude from hemispheric representation to actual numeric values
As noted above, if its West and South values were become negative coordinate values. Chceck the converted longitude and latutude values by ploting in a 2D cartesian space

In [51]:
PD_n['Latitude'] = manage_lat(PD_n)
PD_n['Longitude'] = manage_long(PD_n)
PD_n[["Latitude", "Longitude"]] = PD_n[["Latitude", "Longitude"]].apply(pd.to_numeric)
latlong = PD_n[["Latitude", "Longitude"]]
latlong.head()

## Visalize the data to understand spatiasense

In [53]:
PD_n.head()
names = np.unique(PD_n['Name'])
for name in names:
    dfd = PD_n[PD_n['Name'] == name]
    plt.plot(dfd['Longitude'], dfd['Latitude'], label=name)
plt.show()

## Convert time into 24 hour scale and date into Day of The Year (DOY) format, and inspect the converted date

In [55]:
PD_n['Date'] = [datetime.datetime.strptime(str(PD_n['Date'].iloc[i]),'%Y%m%d').timetuple().tm_yday for i in range(PD_n.shape[0])]
PD_n['Time'] = PD_n['Time']/100   # convert time into proper 24 hourse scale
datetime = PD_n[["Date", "Time"]]
datetime.head()

## Inspect if there is highly outocorrelating attribute in the dataframe
Please note that panda data frame correlation analysis ignores non-numeric columns from correlation computation. This selection of numeric values could also be done mannually. As indicated in the correlation matrix plot, the Minimim pressure and Maximum wind has negatively correlated. As this values are very important for hericane prediction and monitoring I would prefer to keep it.

Remove nodata value and replace with means. here there is no hard rule how to replace no data vale

In [56]:
# replace Nans with column means per hericane based on herican ID
ids = list(np.unique(PD_n['ID']))
for i in range(len(ids)):
    sub = PD_n[PD_n['ID']==ids[i]]
    sub=sub.fillna(sub.drop(columns=['Name', 'Event', 'Status','ID']).mean())
    PD_n[PD_n['ID']==ids[i]] = sub
PD_n.isna().sum()   # check the na removed dataframe

## Plot cross correlation and inspect the spatial and tenporal correlation

In [57]:
crmtx = round(PD_n.drop(columns=['Name', 'Event', 'Status', 'ID']).corr(), 3)
plt.figure(figsize=(22,22))
sbn.heatmap(crmtx, annot=True)
plt.show()

## Understand data distribution
Inspect the histogram/frequency dstribution of each fields/atrributes. As can be seen from the histogram plots, most of the attributes related to wind speed are with left skwed(right tailed) distributions. Latitude Longitude and Time have almost normal distribition.

In [60]:
PD_n.hist(bins=40, figsize=(22,22))
plt.show()

## Inspect the statistics of each numeric column
As can be seen from descriptive statistics there are 5999 rows/data points. Please note that this data points were irrespective of unique hericane ID or name.

In [61]:
PD_n.describe().transpose()

## Normalization (Optional and not implemented)
This is an optional phase. As I am using Latitude and longitude as feature plus label, normalizing this value will affect the semantic meaning/validity of the attribute that, this step is skipped.
The implementation is provided in the function "prepare_samples" below that if needed, can turn the parameter normalize into True

### Convert dataframe to an array wich is suitable for LSTM
As the problem is coined as multivariate sequence to sequence prediction, The data is prepared as follows. Time series observation for each unique herican is converted into a 3-dimensional numpy array of [B,T, A] where B is number of batchs, T is sequence length(time window) to model subsequent event which is 4 and A is number of attributes which is 18.

In train test split and sampling, the following purposive customization is done. When the data is segregated, the number of rows allocated for trainning were not sufficient that, I simply randomly select testing hericanes. As validation samples are only used to track the learning process, I train without validation samples. This all is to use many data points for model training.

But the code used for sampling, generation of trainingand testing samples are commented and can be used when sufficient data is available or if one decided to work on only hericane coordinates.

Deasign:
    Features: all observations within previous time t units(24 hours)
    Label: latitude and longitue at t+1 units
    As indicated in figure below, column A contains all the attributes at time window t while Lat and long are modeeled as labels
    
   ![image-5.png](attachment:image-5.png)

In [21]:
def sampler(pdf, train_test_ratio = [0.7, 0.15, 0.15], strategy='pick', number_test=3):

    '''
    The function takes full datframe and undertakes sample partitioning
    pdf: is the dataframe that contains all hericane trajectories
    train_test_ratio: the ratio used for training, validation and testing respectively
    strategy: how sample is alloted, either using provided ratios or just user defined number
    number_test: if strategy is "pick" the number of test samples reserved for testing
    returns: list of unique hurricane IDs allocatied for trainning, vaidation and testing
    '''

    inds = list(np.unique(pdf['ID']))  # take uniqe hericane IDS

    random.seed(2)   # for reproduciability
    random.shuffle(inds) # shuffle

    if strategy == 'pick':
        test_ids = random.sample(inds, number_test)
        train_ids = list(set(inds).symmetric_difference(set(test_ids)))
        return train_ids, test_ids
    else:

        N = len(inds)

        trn = round(N*train_test_ratio[0])
        vln = round(N*train_test_ratio[1])
        tsn = round(N*train_test_ratio[2])

        train = inds[:trn]         # training hericanes
        valid = inds[trn:trn+vln]  # validation hericanes
        test = inds[trn+vln:]      # test hericanes

        return train, valid, test

In [22]:
def prepare_samples(pdf, hid, remove_cols=['ID', 'Event', 'Status', 'Name'], w=3, normalize=False):
    '''
    Prepares samples by taking specific hurricane ID
    pdf: dataframe containing all attributes and labels
    hid: unique hurricane ID
    remove_cols: list of column/attribute names not included in the analysis
    W: window length to segment time sereis data
    normalize: boolean value whether toundertaken data normalization. If True, undrtakes minmax-normalization
    returns: prepared features with shape [B, T, A] and labels with shape [B, L], where L is number of sequence length to predict
    '''

    samp = pdf[pdf['ID']==hid]  # subset specific hericane

    if remove_cols is not None:
        feat_negin = samp.drop(remove_cols, axis=1)
    else:
        feat_negin = samp
#     feat = feat_negin.drop(['Latitude', 'Longitude'],  axis=1)
    feat = feat_negin

    if normalize:
        min_max_scaler = preprocessing.MinMaxScaler()
        scaled = min_max_scaler.fit_transform(feat.values)
        col_names = list(feat.keys())
        feat = pd.DataFrame(scaled, columns=col_names)

    label = feat_negin[['Latitude', 'Longitude']]

    c = feat.shape[0]
    if c<=w:
        print('Hericane with ID: {} -> has length less than window and not included in the features'.format(hid))
        return None, None
    else:
        feature_map = []
        label_map = []

        for i in range(c-w-1):
            FF = np.array(feat.iloc[i:i+w])
            LL = label.iloc[i+w+1]
            feature_map.append(FF)
            label_map.append(LL)

        feature_map = np.array(feature_map)
        label_map = np.array(label_map)

        # # for more caution
#         if len(feature_map.shape) == 2:
#             feature_map = np.expand_dims(feature_map, axis=0)
#         if len(label_map.shape) == 2:
#             label_map = np.expand_dims(label_map, axis=0)

        return feature_map, label_map

In [23]:
def prepare_features(IDS, pdfs, w=3):
    '''
    Undertakes batch processing for all hurricanes
    IDS: list of unique Hurricane IDs
    pdfs: data frame containing all features and labels columns
    w: window length to segment time series into datapoints suitable for LSTM input
    '''

    features = []
    labels = []

    for ids in IDS:
        data = prepare_samples(pdf=pdfs, hid=ids, w=w)

        if data[0] is None or data[1] is None:
            pass
        else:
            if data[0].shape == 2 or data[1].shape == 2:
                data[0] = np.expand_dimes(data[0], axis=0)
                data[1] = np.expand_dimes(data[1], axis=0)
                features.append(data[0])
                labels.append(data[1])
            else:
                features.append(data[0])
                labels.append(data[1])

    inds = [i for i in range(len(features)) if len(features[i].shape)==3]
    features = [features[j] for j in inds]
    labels = [labels[j] for j in inds]

    features = np.concatenate(tuple(features), axis=0)
    labels = np.concatenate(tuple(labels), axis=0)

    return features, labels

## Select training and test hericanes before data preparation.
This is mainly bacause, the test data should be inspected independently using its unique hericane ID later on after trainning

In [62]:
test_sample_intake = 'pick'
if test_sample_intake == 'pick':
    samples = sampler(PD_n)
    train_ids = samples[0]
    test_ids = samples[1]
    print('Length of training herican samples: {}'.format(len(train_ids)))
    print('Length of test herican samples: {}'.format(len(test_ids)))
else:
    samples = sampler(PD_n, strategy='ratio')
    train_ids = samples[0]
    validation_ids = samples[1]
    test_ids = samples[2]
    print('Length of training herican samples: {}'.format(len(train_ids)))
    print('Length of validation herican samples: {}'.format(len(validation_ids)))
    print('Length of test herican samples: {}'.format(len(test_ids)))

###### Please note that as indicated below if there is herican event with observations less than or equal to proposed sequence length to model, it is excluded from being part of analysis

# Prepare trainning features

In [63]:
train_samples = prepare_features(IDS=train_ids, pdfs=PD_n, w=3)
train_feature = train_samples[0]
train_label = train_samples[1]
print('Shape of training features', train_feature.shape)
print('Shape of trainning labels',  train_label.shape)

## Optional train test split
*Note: the following commented cells are used if we have sufficient amount of samples to partition*

In [ ]:
# Prepare validation samples
# validation_samples = prepare_features(IDS=validation_ids, pdfs=PD_n)
# validation_feature = validation_samples[0]
# validation_label = validation_samples[1]
# validation_feature.shape, validation_label.shape

In [ ]:
# prepare test samples
# test_samples = prepare_features(IDS=test_ids, pdfs=PD_n)
# test_feature = validation_samples[0]
# test_label = validation_samples[1]
# test_feature.shape, test_label.shape

In [ ]:
# combine, shuffle and sample again
# features = np.concatenate((train_feature,validation_feature, test_feature), axis=0)
# labels  = np.concatenate((train_label,validation_label, test_label), axis=0)
# features.shape, labels.shape

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.05, random_state=42)
# X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Model design

In [26]:
# optimizers to use give a try and select one. In principle this should be done in a
# grid-search strategy but I have tried it with one by one basis
# sgd = SGD(learning_rate=0.001, momentum=0.9)
# adm = Adam(learning_rate=0.001)
msprop = RMSprop(learning_rate=0.0001,rho=0.9, momentum=0.9, centered=True)
# import tensorflow as tf


In [64]:
tf.keras.backend.clear_session()
data_shape = (train_feature.shape[1], train_feature.shape[-1])
model = Sequential()
model.add(Bidirectional(LSTM(500,return_sequences=True), input_shape=data_shape))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(500)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Dense(2))
model.compile(loss='mse', optimizer=msprop)
model.summary()
checkpoint_path = '/content/drive/MyDrive/plus-deepdata-social-good-master/plus-deepdata-social-good-master/checkpoint/.weights.h5'
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

#Load pretrained weight

In [28]:
try:
  model.load_weights(checkpoint_path)
  print(f'Pretrained weight loaded from: {checkpoint_path}')
except:
  print('pretrained weight is not loaded, starting training from scratch')

Pretrained weight loaded from: /content/drive/MyDrive/plus-deepdata-social-good-master/plus-deepdata-social-good-master/checkpoint/.weights.h5


# Train the model

In [65]:
hist = model.fit(x=train_feature,
                 y=train_label,
                 epochs=15,
                 verbose=1,
                 callbacks=[cp_callback])

# Test the performance of the model and visalize predictions

In [66]:
# compute metrics and plot predicted and reference hericane trajectories
MSE = tf.keras.metrics.MeanSquaredError()   # mean squared error
MAE = tf.keras.metrics.MeanAbsoluteError()   # mean absolute error

mse = []
mae = []

for i in range(len(test_ids)):
    one_id = test_ids[i]
    tr_f = prepare_samples(pdf=PD_n, hid=one_id, w=3)  # prepare test data and ingest into trained model
    pred = hist.model.predict(tr_f[0])

    MSE.update_state(tf.constant(tr_f[1]), pred)  # compute accuracy per test herican ID usimg predicted and refenrece data
    mse.append(MSE.result().numpy()) # append it
    MSE.reset_state()

    MAE.update_state(tf.constant(tr_f[1]), pred)  # compute accuracy per test herican ID usimg predicted and refenrece data
    mae.append(MAE.result().numpy()) # append it
    MAE.reset_state()

    plt.scatter(pred[:,1], pred[:,0])
    plt.plot(pred[:,1], pred[:,0], label='predicted')
    plt.scatter(tr_f[1][:,1], tr_f[1][:,0])
    plt.plot(tr_f[1][:,1], tr_f[1][:,0], label='observed')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title(one_id)
    plt.legend()
    plt.show()

In [8]:
# check the mean squared error
print('Herican ID -> Mean Absolute Error  -> Mean Squred Error')
for i in range(len(test_ids)):
    print('{} -> {}  -> {}'.format(test_ids[i], mae[i], mse[i]))

## Optional: Save the model weights:
If the model weights are not saved during the training phase the following cell could be unchecked and the model weights cold be save.

In [39]:
# Save the trained model
# name = 'best_trajectory_model_weight.h5'
# if 'trained_model_weights' not in os.listdir(data_folder): # create model weight directory
#     model_path = os.path.join(data_folder, 'trained_model_weights')
#     os.makedirs(model_path, exist_ok=True)
#     model_name = os.path.join(model_path, name)
# else:
# model_path = os.path.join(data_folder, 'trained_model_weights')
# model_name = '/content/drive/MyDrive/plus-deepdata-social-good-master/trained_model_weights/weights.ckpt' # os.path.join(model_path, name)
# model.save_weights('.weights.h5') # save model weights

# **Optional**
If need to load save weights first define the same model and load weights

In [33]:
# tf.keras.backend.clear_session()
# data_shape = (train_feature.shape[1], train_feature.shape[-1])
# new_model = Sequential()
# new_model.add(Bidirectional(LSTM(500,return_sequences=True), input_shape=data_shape))
# new_model.add(BatchNormalization())
# new_model.add(Bidirectional(LSTM(500)))
# new_model.add(BatchNormalization())
# new_model.add(Dropout(0.2))
# new_model.add(Dense(64)) # relu
# new_model.add(Dropout(0.2))   #
# new_model.add(Dense(2)) # out_size
# new_model.compile(loss='mse', optimizer=msprop)

## Load model

In [3]:
# checkpoint_path = '/content/drive/MyDrive/plus-deepdata-social-good-master/plus-deepdata-social-good-master/checkpoint/.weights.h5'
# if checkpoint_path is not None:
#     new_model.load_weights(checkpoint_path)
#     print('model weights loaded from {}'.format(checkpoint_path))

## Perform test and visualize prediction reults

In [1]:
# # compute metrics and plot predicted and reference hericane trajectories
# MSE = tf.keras.metrics.MeanSquaredError()   # mean squared error
# MAE = tf.keras.metrics.MeanAbsoluteError()   # mean absolute error

# mse = []
# mae = []

# for i in range(len(test_ids)):
#     one_id = test_ids[i]
#     tr_f = prepare_samples(pdf=PD_n, hid=one_id, w=3)  # prepare test data and ingest into trained model
#     pred = model.predict(tr_f[0])

#     MSE.update_state(tf.constant(tr_f[1]), pred)  # compute accuracy per test herican ID usimg predicted and refenrece data
#     mse.append(MSE.result().numpy()) # append it
#     MSE.reset_state()

#     MAE.update_state(tf.constant(tr_f[1]), pred)  # compute accuracy per test herican ID usimg predicted and refenrece data
#     mae.append(MAE.result().numpy()) # append it
#     MAE.reset_state()

#     plt.scatter(pred[:,1], pred[:,0])
#     plt.plot(pred[:,1], pred[:,0], label='predicted')
#     plt.scatter(tr_f[1][:,1], tr_f[1][:,0])
#     plt.plot(tr_f[1][:,1], tr_f[1][:,0], label='observed')
#     plt.xlabel('Longitude')
#     plt.ylabel('Latitude')
#     plt.title(one_id)
#     plt.legend()
#     plt.show()

In [2]:
# print('Herican ID -> Mean Absolute Error  -> Mean Squred Error')
# for i in range(len(test_ids)):
#     print('{} -> {}  -> {}'.format(test_ids[i], mae[i], mse[i]))